In [ ]:
import numpy as np
import cv2 as cv
import os
import skimage
from skimage import io, util
from scipy.stats import norm

In [ ]:
!pip list --format=freeze > requirements.txt

In [ ]:
data_path = os.path.join(os.path.dirname(os.path.abspath('Assignment1.ipynb')), 'COL780 Dataset/IBMtest2/input')
output_path = os.path.join(os.path.dirname(os.path.abspath('Assignment1.ipynb')), 'COL780 Dataset/IBMtest2/result')

number_files = len(os.listdir(data_path))
files = []
for i in range(number_files):
    file_name = os.path.join(data_path, 'in' + '%06d' % i + '.png')
    files.append(file_name)
img1 = cv.imread(files[0])
shape = img1.shape
gray1 = np.array([[0 for j in range(shape[1])] for i in range(shape[0])])
print(shape)
for i in range(shape[0]):
    for j in range(shape[1]):
        gray1[i][j] = 0.2989*img1[i][j][0] + 0.5870*img1[i][j][1] + 0.1140*img1[i][j][2]


In [ ]:
K = 4
Tb = 0.8
alpha = 0.5
weight = [0.6, 0.25, 0.1, 0.05]
sigma_s = 225.0

height = shape[0] 
width = shape[1]

class bgd_sub:
    
    def __init__(self):
        self.wt = None
        self.mu = None
        self.sigma_sq = None
        self.result = None
        
    def match(self, pixel, mu, sigma):
        if sigma == 0:
            sigma = 0.01
        return np.absolute((pixel - mu) / np.sqrt(sigma)) <= 2.5
    
    def main(self):
        self.wt = np.array([[weight for j in range(shape[1])] for i in range(shape[0])])
        self.mu = np.array([[[gray1[i][j] for k in range(K)] for j in range(shape[1])] for i in range(shape[0])])
        self.sigma_sq = np.array([[[sigma_s for k in range(K)] for j in range(shape[1])] for i in range(shape[0])])
        self.result = np.array([[0 for j in range(shape[1])] for i in range(shape[0])])
        
        output_no = 0
        for file in files:
            img = cv.imread(file)
            gray = np.array([[0 for j in range(shape[1])] for i in range(shape[0])])

            for i in range(shape[0]):
                for j in range(shape[1]):
                    gray[i][j] = 0.2989*img[i][j][0] + 0.5870*img[i][j][1] + 0.1140*img[i][j][2]

            for i in range(shape[0]):
                for j in range(shape[1]):
                    matched_gaussian = -1
                    for k in range(K):
                        if self.match(gray[i][j], self.mu[i][j][k], self.sigma_sq[i][j][k]):
                            matched_gaussian = k
                            break
                    if matched_gaussian == -1:
                        temp = [self.wt[i][j][k] for k in range(K)]
                        lowest_wt_gaussian = temp.index(min(temp))
                        self.mu[i][j][lowest_wt_gaussian] = gray[i][j]
                        self.sigma_sq[i][j][lowest_wt_gaussian] = sigma_s
                    else:
                        for k in range(K):
                            self.wt[i][j][k] = (1 - alpha) * self.wt[i][j][k]
                        self.wt[i][j][matched_gaussian] += alpha
                        
                        matched_mu = self.mu[i][j][matched_gaussian]
                        matched_sigma_sq = self.sigma_sq[i][j][matched_gaussian]
                        
                        if matched_sigma_sq == 0:
                            matched_sigma_sq = 0.1
                            
                        pdf_val = norm.pdf(gray[i][j] , loc = matched_mu , scale = np.sqrt(matched_sigma_sq))
                        rho = alpha * pdf_val
                        
                        self.mu[i][j][matched_gaussian] = int((1 - rho) * matched_mu + rho * gray[i][j])
                        diff = gray[i][j] - self.mu[i][j][matched_gaussian]
                        diff = diff ** 2
                        self.sigma_sq[i][j][matched_gaussian] = (1 - rho) * matched_sigma_sq + rho * diff
                    
                    for k in range(K):
                        if self.sigma_sq[i][j][k] == 0:
                            self.sigma_sq[i][j][k] = 0.01

                    ratio = self.wt[i][j] / np.sqrt(self.sigma_sq[i][j])
                    order = np.argsort(ratio)
                    order = np.flip(order)
                    total_bkg_wt = 0
                    index = 0
                    while(total_bkg_wt <= Tb):
                        total_bkg_wt += self.wt[i][j][order[index]]
                        index += 1
                                        
                    for t in range(index):
                        if self.match(gray[i][j], self.mu[i][j][order[t]], self.sigma_sq[i][j][order[t]]):
                            self.result[i][j] = 255
                        else:
                            self.result[i][j] = 0
                            break
                            
            cv.imwrite(os.path.join(output_path, '%06d' % output_no +'.png'), self.result)
            print("DONE ", str(output_no))
            output_no += 1

model = bgd_sub()
model.main()        

In [ ]:
s = shape[0] / 15
t = 0.1
output = [] 
for i in range(number_files):
    file_name = os.path.join(output_path, '%06d' % i + '.png')
    output.append(file_name)
path = os.path.join(os.path.dirname(output_path), 'final/')

file_no = 0                    
for file in output:
    img1 = cv.imread(file)
    img = img1[:, :, 0]
    integral = np.array([[int(img[i][j]) for j in range(shape[1])] for i in range(shape[0])])
    
    out = np.array([[255 for j in range(shape[1])] for i in range(shape[0])])
                             
    for i in range(1, shape[0]):
        integral[i][0] += integral[i - 1][0]
    for j in range(1, shape[1]):
        integral[0][j] += integral[0][j - 1]
                             
    for i in range(1, shape[0]):
        for j in range(1, shape[1]):
            integral[i][j] += integral[i - 1][j] + integral[i][j - 1] - integral[i - 1][j - 1]
                             
    for i in range(shape[0]):
        for j in range(shape[1]):
            x1 = int(max(i - s/2, 0))
            x2 = int(min(i + s/2, shape[0] - 1))
            y1 = int(max(j - s/2, 0))
            y2 = int(min(j + s/2, shape[1] - 1))
            count = (x2 - x1) * (y2 - y1)
            sum_my = integral[x2][y2] + integral[x1][y1] - integral[x1][y2] - integral[x2][y1]
            sum_my /= 255
            if sum_my == 0:
                sum_my = 0.1
            if (count - sum_my) / count >= t:
                for x in range(x1 + 1, x2 + 1):
                    for y in range(y1 + 1, y2 + 1):
                        out[x][y] = 0
    cv.imwrite(os.path.join(path, '%06d' % file_no +'.png'), out)
    print(file_no)
    file_no += 1
    

In [ ]:
b = int(shape[0] / 2)
l = int(shape[1] / 7)
print(l, b)
#area = l * b

def IoU(x1, y1, x2, y2):
    a1 = x1 + b
    b1 = y1 + l
    a2 = x2 + b
    b2 = y2 + l
    xx = max(x1, x2)
    yy = max(y1, y2)
    aa = min(a1, a2)
    bb = min(b1, b2)
    
    w = max(0, aa - xx)
    h = max(0, bb - yy)
    return (w * h) / (2 * area - w * h)
    

# Draw parameters
thresh = 0.1


output = [] 
for i in range(number_files):
    file_name = os.path.join(output_path, '%06d' % i + '.png')
    output.append(file_name)
path = os.path.join(os.path.dirname(output_path), 'final/')

file_no = 0                    
for file in output:
    img1 = cv.imread(file)
    img = img1[:, :, 0]
    integral = np.array([[int(img[i][j]) for j in range(shape[1])] for i in range(shape[0])])
    out = img1.copy()
                             
    for i in range(1, shape[0]):
        integral[i][0] += integral[i - 1][0]
    for j in range(1, shape[1]):
        integral[0][j] += integral[0][j - 1]
                             
    for i in range(1, shape[0]):
        for j in range(1, shape[1]):
            integral[i][j] += integral[i - 1][j] + integral[i][j - 1] - integral[i - 1][j - 1]
            
    list_my = []
    scores = []
    
    for i in range(shape[0] - b):
        for j in range(shape[1] - l):
            score = integral[i + b][j + l] - integral[i][j + l] - integral[i + b][j] + integral[i][j]
            score = score / 255
            if score / area <= 0.75:
                list_my.append([i, j])
                scores.append(score)

    scores = np.array(scores)
    order = scores.argsort()
    keep = []
    
    while(len(order) > 0):
        index = order[-1]
        keep.append(list_my[index])
        P = list_my[index]
        order = order[: -1]
        if len(order) == 0:
            break
        mask = []
        for i in range(len(order)):
            box = list_my[order[i]]
            if IoU(P[0], P[1], box[0], box[1]) < 0.05:
                mask.append(True)
            else:
                mask.append(False)
        order = order[mask]
                
    for el in keep:
        cv.rectangle(out, (int(el[1]), int(el[0])), (int(el[1] + l), int(el[0] + b)), (255, 0, 0), 1)
        
    cv.imwrite(os.path.join(path, '%06d' % file_no +'.png'), out)
    print(file_no)
    file_no += 1
    

In [ ]:
factor = 10
output = [] 
for i in range(number_files):
    file_name = os.path.join(output_path, '%06d' % i + '.png')
    output.append(file_name)
path = os.path.join(os.path.dirname(output_path), 'final/')

file_no = 0                    
for file in output:
    img1 = cv.imread(file)
    img = img1[:, :, 0]
    integral = np.array([[int(img[i][j]) for j in range(shape[1])] for i in range(shape[0])])
    
    #out = np.array([[255 for j in range(shape[1])] for i in range(shape[0])])
    
    for i in range(shape[0] // factor):
        for j in range(shape[1] // factor):
            black_count = 0
            for x in range (factor):
                for y in range(factor):
                    if img[i * factor + x][j * factor + y] == 0:
                        black_count += 1
            if black_count >= 50:
                for x in range (factor):
                    for y in range(factor):
                        integral[i * factor + x][j * factor + y] = 0
            else:
                for x in range (factor):
                    for y in range(factor):
                        integral[i * factor + x][j * factor + y] = 255
    cv.imwrite(os.path.join(path, '%06d' % file_no +'.png'), integral)
    print(file_no)
    file_no += 1
    

In [ ]:
import numpy 
a = [[1, 2, 3], [3, 4, 5]]
x = numpy.array(a).shape[0]
print(x)

In [ ]:
import os
import cv2 as cv

data_path = '/Users/silentstorm/SEM6/COL780/COL780 Dataset/HighwayI/final'
files = []
for i in range(440):
    file_name = os.path.join(data_path, '%06d' % i + '.png')
    files.append(file_name)
os.chdir(os.path.dirname(data_path))
img1 = cv.imread(files[10])
width = img1.shape[0]
height = img1.shape[1]

def generate_video():
    video = cv.VideoWriter('HighwayI-part2.avi', cv.VideoWriter_fourcc(*'DIVX'), 15, (height, width))     
    for file in files:
        img = cv.imread(file)
        video.write(img) 
    video.release()

generate_video()